<a href="https://colab.research.google.com/github/Guhan2348519/SPR_labs/blob/main/2348519_SPR_LAB4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from scipy.signal import freqz

# Load a short speech signal (you can replace this with your own recorded signal)
filename = 'path_to_your_speech_file.wav'  # Replace with your own file
signal, sr = librosa.load(filename, sr=None)

# (b) LPC Analysis to extract coefficients
def lpc_analysis(signal, order):
    # Apply LPC using librosa
    lpc_coeffs = librosa.lpc(signal, order)
    return lpc_coeffs

# (c) Reconstruct the signal using LPC coefficients
def reconstruct_signal(lpc_coeffs, signal_length):
    # Reconstructing signal from LPC coefficients
    reconstructed_signal = np.zeros(signal_length)
    for n in range(len(reconstructed_signal)):
        reconstructed_signal[n] = np.dot(lpc_coeffs[1:], reconstructed_signal[n-len(lpc_coeffs[1:]):n][::-1]) - lpc_coeffs[0]
    return reconstructed_signal

# (d) Estimate formants from LPC coefficients
def estimate_formants(lpc_coeffs, sr):
    # Calculate roots of LPC polynomial
    roots = np.roots(lpc_coeffs)
    roots = [r for r in roots if np.imag(r) >= 0]
    angles = np.angle(roots)

    # Convert angles to frequencies (formants)
    formants = sorted(angles * (sr / (2 * np.pi)))
    return formants

# (e) Plot the formant frequencies on a frequency response plot
def plot_formants(lpc_coeffs, sr, formants):
    w, h = freqz(1, lpc_coeffs, worN=512, fs=sr)

    plt.figure(figsize=(10, 6))
    plt.plot(w, 20 * np.log10(np.abs(h)), label='Frequency response')
    for formant in formants:
        plt.axvline(formant, color='red', linestyle='--', label=f'Formant {round(formant, 2)} Hz')
    plt.title('LPC Frequency Response with Formants')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.legend()
    plt.show()

# Parameters
order = 16  # LPC order (adjust based on the signal)

# (b) Get LPC coefficients
lpc_coeffs = lpc_analysis(signal, order)

# (c) Reconstruct the signal
reconstructed_signal = reconstruct_signal(lpc_coeffs, len(signal))

# Plot original and reconstructed signals
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(signal)
plt.title("Original Speech Signal")
plt.subplot(2, 1, 2)
plt.plot(reconstructed_signal)
plt.title("Reconstructed Speech Signal")
plt.show()

# (d) Estimate formants from LPC coefficients
formants = estimate_formants(lpc_coeffs, sr)

# Print estimated formants
print(f"Estimated Formants: {formants}")

# (e) Plot formant frequencies
plot_formants(lpc_coeffs, sr, formants)